In [1]:
# automate install required packages
import subprocess
# import system tools
import sys
import webcolors
import re
import math

In [ ]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)
# df = query_from_file(file_name='query/uncurated.sql', params=params)

def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))
# df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))

In [1]:
# enrich color parser
# to utilize this just import it (assuming its in the local or sub dir)
# import parseColor
# webcolors will help us match strings to RGB values and back
import webcolors
# core python libs
import re
import math
# a sample test of colors
testColors = [
    'red, red/ruby/ raddish',   # parse complex mixes       -->     red
    'ruby-red',                 # parse dashes              -->     red
    'ivory white',              # two similar shades        -->     snow
    'black',                    # simple color              -->     black
    'cyan',                     # an off blue color         -->     cyan
    'grey',                     # grey UK spelling          -->     gray
    'gray',                     # gray CAN/US spelling      -->     gray
    'some-madeup-color',        # no match, return input    -->     some-madeup-color
    'black',                    # another simple test       -->     black
    'noir',                     # french/alt black          -->     noir
    'red green blue'            # a mix of three primes     -->     darkslategray
]
# a regex to split a string on dashes, spaces, slashes, back slashes, and commas
colorSplitRegex = r'-| |/|\\|,'
# G E T   A   C O L O R   N A M E
# get the closest color from an rgb tuple
def closest_colour(requested_colour):
    min_colours = {}
    for key, name in webcolors.CSS3_HEX_TO_NAMES.items():
        r_c, g_c, b_c = webcolors.hex_to_rgb(key)
        rd = (r_c - requested_colour[0]) ** 2
        gd = (g_c - requested_colour[1]) ** 2
        bd = (b_c - requested_colour[2]) ** 2
        min_colours[(rd + gd + bd)] = name
    return min_colours[min(min_colours.keys())]
# get the color name either from webcolors or closest_color
def get_colour_name(requested_colour):
    try:
        closest_name = webcolors.rgb_to_name(requested_colour)
    except ValueError:
        closest_name = closest_colour(requested_colour)
    return closest_name
# G E T   A   C O L O R   V A L U E
# convert a string to an rgb tuple
def parseColor(string):
    try:
        # convert the color string to rgb, default spec is css3
        return webcolors.name_to_rgb(string)
    except Exception:
        pass
# parse raw data for strings we can match on
# if the string has a dash, slash, or space try each color individually
# and average out the RGB values with a simple a+b+c/n forumla
def parseColorData(data):
    # init an empty array for color values
    colorValues = []
    # split it on dashes, spaces, slashes, or commas
    if re.split(colorSplitRegex, data):
        for color in re.split(colorSplitRegex, data):
            colorValues.append(parseColor(color))
    # if it can't be split just full send it
    else:
        colorValues.append(parseColor(color))
    # remove None values
    colorValues = [
        colorValue for colorValue in colorValues if colorValue is not None]
    if not len(colorValues):
        return data
    # find the average color and convert it to a color name
    #######################################################
    # init an average color value
    averageColor = [0, 0, 0]
    for colorValue in colorValues:
        for index, value in enumerate(colorValue):
            averageColor[index] += value
    averageColor = [math.floor(value / (len(colorValues) or 1))
                    for value in averageColor]
    # just change this to a return to run this function
    return get_colour_name(averageColor)
# testing
#########
for color in testColors:
    print(parseColorData(color))

red
red
snow
black
cyan
gray
gray
some-madeup-color
black
noir
darkslategray


In [ ]:
######### These are for values that have multiple numbers###############

# Finds the first digit (basically skips over words and letters to find the first number)
def find_first_digit(s, non=False):
    for i, x in enumerate(s):
        if x.isdigit() ^ non:
            return i
    return -1
# i finds the first digit: if it equals zero then there is not a digit, if it equals the end then leave it
def split_digits(s, case=False):
    non = True
    while s:
        i = find_first_digit(s, non)
        if i == 0:
            non = not non
        elif i == -1:
            yield int(s) if s.isdigit() else s if case else s.lower()
            s = ''
        else:
            x, s = s[:i], s[i:]
            yield int(x) if x.isdigit() else x if case else x.lower()

            
# sort the number naturally so 5 will come before 15
def natural_key(s, *args, **kwargs):
    return tuple(split_digits(s, *args, **kwargs))
# x['matches']= x['matches'].apply(lambda x: sorted(x, key=natural_key))


# atof and natural_keys: Will naturally sort
def atof(text):
    try:
        retval = float(text)
    except ValueError:
        retval = text
    return retval

def natural_keys(text):
    '''
    alist.sort(key=natural_keys) sorts in human order
    http://nedbatchelder.com/blog/200712/human_sorting.html
    (See Toothy's implementation in the comments)
    float regex comes from https://stackoverflow.com/a/12643073/190597
    '''
    return [ atof(c) for c in re.split(r'[+-]?([0-9]+(?:[.][0-9]*)?|[.][0-9]+)', text) ]
#########################################################################



# # Replace specific text in a string: Exmapel1  to  Example1
# def replace_all(text, dic):
#     for i, j in dic.items():
#         text = text.replace(i, j)
#     return text
# # replacement={'83/83':'1',    }
# # df= df.apply(
# #     lambda x: [replace_all(string, replacement) for string in x] if type(x)==list else x)

def replace_all(text, dic):
    '''
    Given a text and replacement dictionary, performs multiple str.replace()
    Arguments:
    ==========
    text: str, text to perform str.replace()
    dic: dict, where each value is the string to replace, and each key is the string to replace with
    '''
    for key, value in dic.items():
        for v in value:
            text = text.replace(v, key)
    return text
# power['min_temp_sj']=power['min_temp_sj'].apply(lambda x: replace_all(x,{'':['Ã‚Â°F']}))


# Convert a fraction that does not have an int in front of the fraction "1/3"
def remove_frac(numbers):
    if str(numbers).find('/') > -1:  
        nom, den = numbers.split('/')
        tmp = float(nom)/float(den)
        return tmp
    else:
        return numbers
     
    
    
# Convert a fraction that is seperated by a space "1 1/3"
def remove_fraction(numbers):
    try:
        if numbers.find('/') > -1:   
            inte, fract = numbers.split(' ')
            nom, den = fract.split('/')
            tmp = float(nom)/float(den)
            num = int(inte) + tmp
            return num
        else:
            return numbers
    except:
        return numbers
    


#Convert a fraction that is seperated by a dash "1-1/2"
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        print(numbers)
        inte, fract = numbers.split('-')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers
# tubes_new['height_new'] = tubes_new['height_new'].apply(lambda x: remove_fraction(x))



####Converts the number to a decimal if it has a decimal number but leaves off .0 if it is an int. Be sure to conver it back to a string and add -"in"
def remove_exponent(num):
    return num.to_integral() if num == num.to_integral() else num.normalize()
# from decimal import *
# TWOPLACES = Decimal(10) ** -2
# tubes_new['height_new']=tubes_new['height_new'].astype(float).apply(lambda x: remove_exponent(Decimal(x).quantize(TWOPLACES)))

In [ ]:
## Remove Duplicate values: "['Induction, 'Induction']" to Induction
# matches['matches']=[list(set(sublist)) for sublist in matches['matches']]

In [ ]:
## Split arrays
def split_col(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
    data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [ ]:
## Drop all words and irregular characters
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [ ]:
## Replace blanks with None or 0 to manipulate it
def replace_none(data):
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
    data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [ ]:
## Add Quotes
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")

In [ ]:
## Find Min and Max
def min_max(data):
    data['length']=data['length'].astype(float)
    data['width']=data['width'].astype(float)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [ ]:
## Use KewWordProcessor to find a list of values for a single bucket 
# df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
# df_strategy
# list_values=df_strategy['value'].to_list()
# list_values

# kwp=KeywordProcessor()
# print(kwp.add_keywords_from_list(list_values))
# # Extract Key Words
# df['long_desc'].apply(lambda x: kwp.extract_keywords(x)) #

# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df['matches'].explode().value_counts() 

In [ ]:
# # range function builder
# string = range_string.replace(metric, '')
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

# matches['matches']=matches['matches'].astype(float)
# matches[curation_col] = matches['matches'].apply(range_app)
# matches

In [ ]:
# # Manipulate Arrays
# #import regex as re
# matches[curation_col]=matches['matches'].astype(str)
# matches[curation_col]=matches[curation_col].apply(lambda x: f'"{x}"')
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r"\'",'"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\" ','"',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'\, ',',',str(measure)))
# matches[curation_col]=matches[curation_col].apply(lambda measure: re.sub(r'(?m)^\h*\K"|"(?=\h*$)','',str(measure)))
# matches

In [ ]:
## This is outdated I think
# a=df['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# df['length']=c
# df

In [ ]:
# #Send to the final folder
# def looks_good(customer, attribute, df, matches):
#     drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
# #     non_matches = df[df['matches'].astype(str) == '[]']
# #     non_matches[curation_col] = r'n/a'
# #     non_matches.to_csv(f'{drive_path}/Motion - {attribute} - na upload.csv',index=False)
#     matches.to_csv(f'{drive_path}/Motion - {attribute} - match upload_Shawn.csv',index=False)
# looks_good('Motion Industries', attribute, df, wipe)